This notebook is used to set up the data needed for our project. This notebook will both load and preprocess the data, and finally also split it into a training and test set.\
First we load the data.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

from isoweek import Week

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# load raw data from csv file
my_data_path = "../df_matches.csv"
df = pd.read_csv(my_data_path, sep=',')

df = df.dropna(subset=['match_date','match_conceded', 'coach1_CR', 'coach2_CR'])

# define which columns we wish to keep
relevant = ["match_id", "division_name", "match_date", "match_time", "match_conceded",
            "team1_coach_id", "team1_race_name", "team2_coach_id", "team2_race_name",
            "team1_score", "team2_score","team1_win","team2_win","mirror_match",
            "coach1_ranking","coach2_ranking","coach1_CR","coach2_CR", "team1_roster_id", "team2_roster_id"]

irrelevant_columns = {
    'tournament_name', 'week_year', 'cr_diff2', 'team1_cas_bh',
    'group_name', 'team2_foul', 'team2_id', 'coach2_CR_bin',
    'team1_id', 'year', 'week_date', 'week_number', 'replay_id',
    'team1_cas_rip', 'tv_diff', 'team2_block', 'coach1_CR_bin',
    'has_sp', 'team2_value', 'tournament_type', 'team1_value',
    'team2_comp', 'team1_rush', 'team1_foul', 'team2_inducements',
    'tv_diff2', 'tournament_start', 'team2_cas_rip', 'team1_inducements',
    'tv_bin', 'scheduler', 'team1_comp', 'cr_diff2_bin', 'group_id',
    'division_id', 'tournament_id', 'Unnamed: 0', 'team2_cas_si',
    'current_ruleset', 'team2_cas', 'team2_rush', 'team1_block',
    'tournament_end', 'team2_pass', 'team1_cas',
    'CR_diff', 'team1_cas_si', 'team2_cas_bh', 'team1_pass', 'tv_match',
    'tv_bin2', 'tournament_progression'
}


After having loaded the dataset and defined which columns we wish to use for the project, we start preprocessing the dataset.\
We keep only the relevant columns and only use matches from the competitive division. We also fix the types in the different columns of the dataset.

In [8]:
df_matches_competitive = df[df['division_name'] == 'Competitive']
df_matches = df_matches_competitive[relevant]

In [9]:
# convert object dtype columns to proper pandas dtypes datetime and numeric
df_matches['match_date'] = pd.to_datetime(df_matches.match_date) # Datetime object

# calculate match score difference
df_matches['team1_win'] = np.sign(df_matches['team1_score'] - df_matches['team2_score'])
df_matches['team2_win'] = np.sign(df_matches['team2_score'] - df_matches['team1_score'])

# mirror match
df_matches['mirror_match'] = 0
df_matches.loc[df_matches['team1_race_name'] == df_matches['team2_race_name'], 'mirror_match'] = 1

df_matches = df_matches.dropna(subset=['match_date'])

df_matches['week_number'] = df_matches['match_date'].dt.isocalendar().week

# cannot serialize numpy int OR Int64 when writing HDF5 file, so we go for plain int as all NAs are gone now
df_matches['week_number'] = df_matches['week_number'].fillna(0).astype(int)

# add year based on match ISO week
df_matches['year'] = df_matches['match_date'].dt.isocalendar().year.astype(int)

df_matches['week_year'] = df_matches['year'].astype(str) + '-' + df_matches['week_number'].astype(str)

# use a lambda function since isoweek.Week is not vectorized 
df_matches['week_date'] = pd.to_datetime(df_matches.apply(lambda row : Week(int(row["year"]),int(row["week_number"])).monday(),axis=1))

After the last cleanup, we condence the dataframe even more and only keep the columns we need for the models.

In [10]:
#The selected columns from the competitive dataframe 
selected_columns = [
    'week_date', 
    'match_time',
    'team1_coach_id', 
    'team2_coach_id',
    'team1_win', 
    'team2_win', 
    'team1_race_name', 
    'team2_race_name'
]

#final dataframe 
new_df = df_matches[selected_columns].copy()
new_df['team1_race_name'] = new_df['team1_race_name'].astype('category')
new_df['team2_race_name'] = new_df['team2_race_name'].astype('category')

Now we save new csv files for the cleaned up data, and when split into training and test data.

In [ ]:
#Store the cleaned data in your local directory 
new_df.to_csv('../df_matches_clean.csv', index=False)

X_train, X_test = train_test_split(new_df, test_size=0.2, random_state=42, shuffle=False)

dir_name = "../data"

# Check if the directory exists
if not os.path.exists(dir_name):
    # Create the directory
    os.makedirs(dir_name)

X_train.to_csv("../data/X_train.csv", index=False)
X_test.to_csv("../data/X_test.csv", index=False)